In [2]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.2/804.2 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

In [4]:
# YOLOv8-poseモデルの読み込み
model = YOLO('yolov8n-pose.pt')

100%|██████████| 6.52M/6.52M [00:00<00:00, 111MB/s]


In [ ]:
import torch

def extract_keypoints(frames):
    all_keypoints = []
    device = torch.device('mps')  # MPSデバイスを指定
    for frame in frames:
        frame = preprocess_frame(frame)
        frame = torch.tensor(frame, dtype=torch.float32).to(device)  # フレームをGPUに転送
        results = model(frame)

        if len(results) > 0 and len(results[0].keypoints) > 0:
            keypoints_list = results[0].keypoints
            bboxes = results[0].boxes  # バウンディングボックスのリスト

            if bboxes is not None and len(bboxes) > 0:
                # バウンディングボックスの面積を計算
                try:
                    xyxy = bboxes.xyxy.cpu().numpy()  # バウンディングボックスの座標を取得
                    areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in xyxy]
                except IndexError as e:
                    print(f"Error calculating areas: {e}")
                    print(f"Bounding boxes: {bboxes}")
                    continue

                # 最も大きいバウンディングボックスのインデックスを取得
                max_area_index = np.argmax(areas)

                # 最も大きいバウンディングボックスに対応するキーポイントを取得
                keypoints = keypoints_list[max_area_index].xy[0].cpu().numpy()

                all_keypoints.append(keypoints)
            else:
                print(f"Warning: No bounding boxes found in frame.")
        else:
            print(f"Warning: No keypoints found in frame.")

    return all_keypoints  # np.arrayの変換を削除


def normalize_keypoints(keypoints):
    hip_index = 11  #  index of left hip
    shoulder_index = 5  # index of shoulder

    normalized_keypoints = []
    for frame_keypoints in keypoints:
        if len(frame_keypoints) > max(hip_index, shoulder_index):
            hip_point = frame_keypoints[hip_index]
            shoulder_point = frame_keypoints[shoulder_index]

            # 腰のポイントを原点(0,0)とし、他のポイントを相対位置として計算
            relative_points = frame_keypoints - hip_point

            # スケーリング、モデルをロバストにするため
            scale_factor = np.linalg.norm(shoulder_point - hip_point)
            if scale_factor != 0:
                relative_points /= scale_factor

            normalized_keypoints.append(relative_points)
        else:
            print(f"Warning: Frame with insufficient keypoints detected. Skipping this frame.")

    return np.array(normalized_keypoints)

def process_folder(folder_path, label):
    print(f"Processing folder: {folder_path}")
    video_data = []
    for video_file in os.listdir(folder_path):
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(folder_path, video_file)
            frames = process_video(video_path)
            keypoints = extract_keypoints(frames)
            if len(keypoints) > 0:
                normalized_keypoints = normalize_keypoints(keypoints)
                if len(normalized_keypoints) > 0:
                    video_data.append((normalized_keypoints, label))
                else:
                    print(f"Warning: No valid keypoints found in video {video_file}")
            else:
                print(f"Warning: No keypoints detected in video {video_file}")
    return video_data

    # デバッグ情報の出力
    print(f"Number of videos processed: {len(video_data)}")

# 全てのショットタイプを処理
shot_types = ['forehand_stroke','forehand_slice','forehand_volley', 'backhand_stroke', 'backhand_volley', 'backhand_slice']
all_data = []
for label, shot_type in enumerate(shot_types):
    folder_path = f'/Users/yusuke.s/Documents/pickleball_videos/{shot_type}'
    all_data.extend(process_folder(folder_path, label))

Processing folder: /Users/yusuke.s/Documents/pickleball_videos/forehand_stroke



0: 640x640 1 person, 488.9ms
Speed: 42.5ms preprocess, 488.9ms inference, 6.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 357.8ms
Speed: 3.3ms preprocess, 357.8ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 375.7ms
Speed: 39.1ms preprocess, 375.7ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 306.2ms
Speed: 5.8ms preprocess, 306.2ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 456.7ms
Speed: 3.4ms preprocess, 456.7ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 381.2ms
Speed: 2.5ms preprocess, 381.2ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 903.8ms
Speed: 3.9ms preprocess, 903.8ms inference, 31.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1260.9ms
Speed: 43.9ms preprocess, 1260.9ms inference, 54.5ms postprocess per 

Processing folder: /Users/yusuke.s/Documents/pickleball_videos/forehand_slice


0: 640x640 2 persons, 398.6ms
Speed: 2.5ms preprocess, 398.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 297.7ms
Speed: 2.1ms preprocess, 297.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 298.1ms
Speed: 2.2ms preprocess, 298.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 297.8ms
Speed: 2.1ms preprocess, 297.8ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 324.8ms
Speed: 2.2ms preprocess, 324.8ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 297.3ms
Speed: 2.1ms preprocess, 297.3ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 293.2ms
Speed: 1.9ms preprocess, 293.2ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 312.0ms
Speed: 2.2ms preprocess, 312.0ms inference, 3.3ms postprocess per i

Processing folder: /Users/yusuke.s/Documents/pickleball_videos/forehand_volley


...

0: 640x640 2 persons, 313.3ms
Speed: 2.0ms preprocess, 313.3ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 293.5ms
Speed: 2.1ms preprocess, 293.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 289.8ms
Speed: 1.9ms preprocess, 289.8ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 283.5ms
Speed: 2.1ms preprocess, 283.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 297.2ms
Speed: 2.1ms preprocess, 297.2ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 294.1ms
Speed: 2.2ms preprocess, 294.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 574.2ms
Speed: 2.5ms preprocess, 574.2ms inference, 5.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 374.7ms
Speed: 3.0ms preprocess, 374.7ms inference, 3.2ms postprocess per ima

Processing folder: /Users/yusuke.s/Documents/pickleball_videos/backhand_slice


0: 640x640 1 person, 305.7ms
Speed: 2.1ms preprocess, 305.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 281.5ms
Speed: 2.0ms preprocess, 281.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 279.9ms
Speed: 1.9ms preprocess, 279.9ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 281.9ms
Speed: 2.0ms preprocess, 281.9ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.0ms
Speed: 2.0ms preprocess, 280.0ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 279.9ms
Speed: 2.0ms preprocess, 279.9ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.0ms
Speed: 1.9ms preprocess, 280.0ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.8ms
Speed: 2.1ms preprocess, 280.8ms inference, 2.9ms postprocess per image at 

0: 640x640 1 person, 337.1ms
Speed: 1.9ms preprocess, 337.1ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 282.7ms
Speed: 1.9ms preprocess, 282.7ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.9ms
Speed: 2.0ms preprocess, 280.9ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 279.5ms
Speed: 2.0ms preprocess, 279.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 282.4ms
Speed: 2.0ms preprocess, 282.4ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.5ms
Speed: 1.9ms preprocess, 280.5ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 282.4ms
Speed: 2.0ms preprocess, 282.4ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.7ms
Speed: 2.0ms preprocess, 280.7ms inference, 2.8ms postprocess per image at 

0: 640x640 1 person, 282.0ms
Speed: 2.2ms preprocess, 282.0ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 282.6ms
Speed: 2.0ms preprocess, 282.6ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 282.8ms
Speed: 2.0ms preprocess, 282.8ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.0ms
Speed: 2.3ms preprocess, 280.0ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 281.3ms
Speed: 2.0ms preprocess, 281.3ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 279.4ms
Speed: 2.0ms preprocess, 279.4ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 280.7ms
Speed: 1.9ms preprocess, 280.7ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 279.4ms
Speed: 2.0ms preprocess, 279.4ms inference, 2.9ms postprocess per image at 

In [ ]:
"""import numpy as np
# Convert to numpy array
all_data_array = np.array(all_data, dtype=object)
# Save to .npy file
np.save('all_data.npy', all_data_array)"""

"import numpy as np\n# Convert to numpy array\nall_data_array = np.array(all_data, dtype=object)\n# Save to .npy file\nnp.save('all_data.npy', all_data_array)"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
all_data = np.load('/content/drive/MyDrive/all_data.npy', allow_pickle=True)

In [7]:
def pad_sequences(sequences, max_length):
    # シーケンスの長さを揃える
    return [seq[:max_length] if len(seq) > max_length else np.pad(seq, ((0, max_length - len(seq)), (0, 0), (0, 0)), 'constant') for seq in sequences]

# データの整形
X = [data[0] for data in all_data]
y = [data[1] for data in all_data]

# シーケンスの長さを揃える（例：最大150フレーム）
X_padded = pad_sequences(X, 150)

# numpy配列に変換
X_array = np.array(X_padded)
y_array = np.array(y)

# one-hot エンコーディング
shot_types = ['forehand_stroke','forehand_slice','forehand_volley', 'backhand_stroke', 'backhand_volley', 'backhand_slice']
y_onehot = np.eye(len(shot_types))[y_array]

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import Sequence
import tensorflow as tf

In [13]:
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        # 形状を変更: (batch_size, 150, 17, 2) -> (batch_size, 150, 34)
        batch_x_reshaped = batch_x.reshape(batch_x.shape[0], batch_x.shape[1], -1)
        return batch_x_reshaped, batch_y


In [15]:
def create_model(sequence_length, num_keypoints, num_coords, num_classes):
    model = Sequential([
        TimeDistributed(Flatten(), input_shape=(sequence_length, num_keypoints, num_coords)),
        LSTM(64, return_sequences=True),
        LSTM(32),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# トレーニング、検証、テストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_array, y_onehot, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# データジェネレータの作成
train_generator = DataGenerator(X_train, y_train, batch_size=16)
val_generator = DataGenerator(X_val, y_val, batch_size=16)
test_generator = DataGenerator(X_test, y_test, batch_size=16)

# モデルの作成
sequence_length = 150
num_keypoints = 17
num_coords = 2
num_classes = 6
model = create_model(sequence_length, num_keypoints, num_coords, num_classes)

# Early Stoppingの設定
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# モデルのトレーニング
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# モデルの評価
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")

Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    ValueError: Ex